<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>
<div style="float:left; width:98 px; height:100px;">
<img src="http://seasonedchaos.github.io/assets/img/SC_logo.jpg" alt="Seasoned Chaos logo" style="height: 98px;">
</div>
    
<h1 style="margin-left: 120px;"> Bouncing balls in a ciruclar boundary </h1>

<p style="margin-left: 120px;">   By: Kayla Besong, Seasoned Chaos team </p>
<p style="margin-left: 120px;">   This code uses basic kinematic equations to illustrate how the trajectory of two balls bouncing within a circular boundary differs with only slight offset in initial x-position. The purpose is to highlight chaos theory and how slightly different initial conditions can lead to wildly different results. </p>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

## Equations to simulate a bouncing ball

### Basic Kinematics

# $\Delta x$ =  $v_{0}t$ + $\left(\frac{1}{2}\right)$$at^{2}$ 
# $v$ = $v_{0}$ + $at$ 

### Reversal at circular boundary

#### unit normal vectors:
### $n_{x}$ =  $\left(\frac{x_{i-1}}{\sqrt{x_{i-1}^{2} + y_{i-1}^{2}}}\right)$ $n_{y}$ =  $\left(\frac{y_{i-1}}{\sqrt{x_{i-1}^{2} + y_{i-1}^{2}}}\right)$

#### unit normal vectors used to reverse direction of velocity and account for 'overshoot' at boundary

### $NewVelocity_x$ =  $(n_{y}^{2} - n_{x}^{2})*velocity_{x} - 2n_{x}*n_{y}*velocity_{y}$
### $NewVelocity_y$ =  $-2n_{x}*n_{y}*velocity_{x} + (n_{x}^{2} - n_{y}^{2})*velocity_{y}$

In [3]:
## these are the required libraries, update your env as needed 

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint 
from matplotlib import animation
from IPython.display import HTML


In [4]:
def bounce_solve(t, xy_start, params, bounds):
    
    ''' This function uses basic physics and trigonometry to itegrate how the trajectory of a ball changes over length t,
    given the initial x and y input locaitons, parameters, and scale of boundary. 
    
    
    Input Variables:

        t:          np.linespace representing a time interval over which to compute the trajectory of the ball 

        xy_start:   Initial x,y positions of the ball. 

        params:     Packed list of input float parameters containing gravity, x_acceleration, x_velocity_i, y_velocity_i in that order. 

        bouonds:    List of two numpy arrays (i.e. one x, one y) that define the x-y phase space in 2D form. Variable is set once 
                    in initial condidtions and used to set plotting space, hence the arrays. The first value of each array is used
                    to set the radius of the circular boundary. 
    
    Output:
        
        Tuple containing two arrays, one the x position the other the y, representing the trajectory position over length of input t. 
    
    '''

    x = np.zeros(np.shape(t))
    y = np.zeros(np.shape(t))
    r = np.zeros(np.shape(t))

    gravity, x_acceleration, x_velocity_i, y_velocity_i = params  
    x_lim, y_lim = bounds 
    x_start, y_start = xy_start

    x[0] = x_start
    y[0] = y_start
    
    r[0] = np.sqrt((x_start**2) + (y_start**2))
    
    R = (abs(x_lim[0])+abs(y_lim[0]))/2
    
    dt = t[1] - t[0]

    for i in range(1,len(t)):
        
        # basic kinematic equations
                                
        dy = y_velocity_i*dt + 0.5*gravity*(dt**2)
        dx = x_velocity_i*dt + 0.5*x_acceleration*(dt**2)
        
        x[i] = x[i-1]+dx
        y[i] = y[i-1]+dy

        r[i] = np.sqrt((x[i]**2) + (y[i]**2))
        
        # if collision, manually update the above calculated position 
        
        if r[i] > R:
            
            # collision coordinates, previous position moving towards boundary
            
            xc = x[i-1]
            yc = y[i-1]
            
            # unit vector normal to circle
            
            nx = xc/np.sqrt((xc**2)+(yc**2))
            ny = yc/np.sqrt((xc**2)+(yc**2))

            # reflected velocity
            
            vxc = x_velocity_i
            vyc = y_velocity_i
            
            x_velocity_i = (ny**2 - nx**2)*vxc - 2*nx*ny*vyc
            y_velocity_i = -2*nx*ny*vxc + (nx**2 - ny**2)*vyc

            ## update new dy, dx, x/y position, r for current timestep
            
            dy = y_velocity_i*dt + 0.5*gravity*(dt**2)
            dx = x_velocity_i*dt + 0.5*x_acceleration*(dt**2)            
            
            x[i] = x[i-1]+dx
            y[i] = y[i-1]+dy

            r[i] = np.sqrt((x[i]**2) + (y[i]**2))


        ## update velocity for next step timestep = [i+1] using basic kinematics 
        
        y_velocity_i = y_velocity_i + gravity*dt
        x_velocity_i = x_velocity_i + x_acceleration*dt     
         
        

    return x, y
    
    

In [26]:
type(sols[0])

tuple

## Initial Conditions 

In [23]:

# physics 

gravity = -9.81
x_acceleration = 0
x_velocity_i = 10
y_velocity_i = -10

# bundle for function import 

params = [gravity, x_acceleration, x_velocity_i, y_velocity_i]

# initial positions

start = 0
x_offset = 0.012 

ball1 = [start, start]
ball2 = [start+x_offset, start]
balls = [ball1, ball2]

# boundary to contain balls

x_lim = np.arange(-30, 31)
y_lim = np.arange(-30, 31)

bounds = [x_lim, y_lim]

# 'time' initizaliation or number of steps/intervals to take 

tmax = 25       # controls how long of path/trajectory when animated
Nt = 1000       # bigger Nt = smaller dt = advantagous, if too small Nt, the interval step is too large and boundary is overshot between steps 

t = np.linspace(start, tmax, Nt)



## Solve 

In [24]:
sols = [bounce_solve(t, ball, params, bounds) for ball in balls]

## Plot
#### This set of animation plotting code has been adapted from Kelsey Malloy's Intro_Lorenz_Butterfly which can be found from the same SC post this notebook was curated for

In [7]:
# matplotlib plotting styles for each ball and its path

ens_colors = ['royalblue','limegreen'] 
ens_line_colors = ['blue', 'green']
ens_markers = ['X','X']
marker_sizes=[26,26]
line_widths = [2,2]

csfont = {'fontname':'DIN Alternate'} # Seasoned Chaos font of choice 

In [8]:
## set the data for the animaiton plotting function according to the number of frames defined 

def drawframe2d(n):
    for k in range(0,len(sols)):
        X,Y = sols[k]
        pts[k].set_data(X[n],Y[n])
        lines[k].set_data(X[:n],Y[:n])
    return pts + lines

In [25]:
# Setup figure
fig, ax1 = plt.subplots(figsize=(12,12))
fig.suptitle(f'Chaos Theory & Kinematics: X-offset = {x_offset}', **csfont,fontsize=24, y = 0.93)

#Setting the axes properties

ax1.set_xlim((x_lim[0]-1, x_lim[-1]+1))            
ax1.set_ylim((y_lim)[0]-1, y_lim[-1]+1)
ax1.tick_params(axis='both', labelsize=12)
ax1.annotate('\u00a9 Seasoned Chaos',**csfont,xy=(19,-30),fontsize=16)
ax1.grid(False)

# plot the ciruclar boundary

theta = np.linspace(0, 2*np.pi, 100)
R = (abs(x_lim[0])+abs(y_lim[0]))/2        # this R is calculated the same as in the function from which the collision boundary for the balls is set from. To adjust sizes alter x and y limits in initial conditions. 
x1 = R*np.cos(theta)
x2 = R*np.sin(theta)

ax1.plot(x1, x2, lw = 4, c = 'k')
ax1.set_aspect(1)

# create empty points and lines based on # of balls

pts = sum([ax1.plot([],[],marker=ens_markers[k],color=ens_colors[k],ms=20) for k in range(0,len(sols))],[])
lines = sum([ax1.plot([],[],color=ens_line_colors[k],lw=line_widths[k]) for k in range(0,len(sols))],[])

# more attributes for control 
plt.subplots_adjust(top=0.9,bottom=0.1,left=0.1,right=0.9) 
plt.close()

# create animiation object and render in HTML video
anim = animation.FuncAnimation(fig, drawframe2d, frames=np.arange(0,len(t)), interval=10, blit=False)
HTML(anim.to_html5_video())

In [27]:
anim.save('BouncingBalls_Kinematics.mp4') # save

#### Sources and links helpful to creating this notebook:

##### inspiration and adapted into python from: https://twitter.com/matthen2/status/1393588039742590978
##### animation plots: matplotlib, Kelsey Malloy
##### Circular boundary mathematics: https://math.stackexchange.com/questions/562835/bouncing-of-a-ball-from-circular-boundary, https://developer.amazon.com/blogs/appstore/post/a07ab562-0609-4519-a2ba-9b15d69ea62b/introduction-to-game-math-raw-and-cooked